Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

---

In [1]:
from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import normalize


def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
data = load_diabetes()
X, y = data.data, data.target

In [3]:
def mse_loss_vectorized(w, b, X, y):
    """
    MSE loss function WITHOUT FOR LOOPs , NO REGULARIZATION
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    - gradient with respect to bias b
    """
    loss = 0.0
    dw = np.zeros_like(w)
    
    # YOUR CODE HERE
    m = X.shape[0]
    
    #dw
    dw = -X.T.dot(y- (X.dot(w) + b))
    
    #db
    db = (-1/m)*(np.sum(y - (X@w + b)))
    
    #loss
    loss = (0.5/m)*(y - (np.dot(X,w) + b)).T.dot(y - (np.dot(X,w) + b))

    return loss, dw, np.array(db).reshape(1,)

In [4]:
def soft_threshold(x, delta):
    # YOUR CODE HERE 
     if x > 0.0 and delta < abs(x):
        return x - delta
     elif x < 0.0 and delta < abs(x):
        return x + delta
     else:
        return 0.0

# Lasso Subgradient Descent

In [5]:
def l1_subgradient(w):
    """
    Subgradient of the L1 loss
    """
    dw = np.zeros_like(w)
    # YOUR CODE HERE
    #"""https://ee227c.github.io/code/lecture4.html"""
    dw[ w > 0.] = 1.0
    dw[w < 0.] = -1.0
    
    return dw
    

def lasso_subgradient_mse_loss_vectorized(w, b, X, y, alpha):
    """
    MSE loss function adding the subgradient for w
    """
    loss, dw, db = mse_loss_vectorized(w, b, X, y)
    # Add the subgradient to dw
    # YOUR CODE HERE
    dw -= alpha*l1_subgradient(w)
    return loss, dw, db

In [6]:
class LassolSubgradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice,:]
            y_batch = y[choice]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # YOUR CODE HERE
            self.w = self.w - learning_rate * dw
            self.b = self.b - learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
    
    
    def predict(self, X):
        # YOUR CODE HERE
        return X@self.w + self.b
        
    def loss(self, X_batch, y_batch):
        return lasso_subgradient_mse_loss_vectorized(self.w, self.b, X_batch, y_batch, self.alpha)

In [7]:
model = LassolSubgradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 14967.029597
iteration 10000 / 200000: loss 1376.791169
iteration 20000 / 200000: loss 1366.143796
iteration 30000 / 200000: loss 1471.742015
iteration 40000 / 200000: loss 1529.706626
iteration 50000 / 200000: loss 1315.018005
iteration 60000 / 200000: loss 1435.839680
iteration 70000 / 200000: loss 1497.426635
iteration 80000 / 200000: loss 1288.874577
iteration 90000 / 200000: loss 1443.260950
iteration 100000 / 200000: loss 1420.882908
iteration 110000 / 200000: loss 1443.866108
iteration 120000 / 200000: loss 1356.703933
iteration 130000 / 200000: loss 1479.992635
iteration 140000 / 200000: loss 1486.527776
iteration 150000 / 200000: loss 1532.219350
iteration 160000 / 200000: loss 1242.267435
iteration 170000 / 200000: loss 1355.711536
iteration 180000 / 200000: loss 1483.244150
iteration 190000 / 200000: loss 1527.738380
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2860.0183446682395


In [8]:
model = LassolSubgradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 14348.288153
iteration 10000 / 200000: loss 1434.005624
iteration 20000 / 200000: loss 1433.728920
iteration 30000 / 200000: loss 1372.480914
iteration 40000 / 200000: loss 1343.970349
iteration 50000 / 200000: loss 1576.441652
iteration 60000 / 200000: loss 1509.107407
iteration 70000 / 200000: loss 1479.646915
iteration 80000 / 200000: loss 1343.469659
iteration 90000 / 200000: loss 1289.267340
iteration 100000 / 200000: loss 1439.210068
iteration 110000 / 200000: loss 1386.864419
iteration 120000 / 200000: loss 1308.904845
iteration 130000 / 200000: loss 1583.017535
iteration 140000 / 200000: loss 1564.501024
iteration 150000 / 200000: loss 1417.884652
iteration 160000 / 200000: loss 1431.557229
iteration 170000 / 200000: loss 1443.508218
iteration 180000 / 200000: loss 1565.031677
iteration 190000 / 200000: loss 1438.617900
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 2879.9876316283967


# Lasso Proximal Gradient Descent (iterative soft thresholding algorithm or ISTA)

In [9]:
class LassoProximalGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice,:]
            y_batch = y[choice]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # PROJECT THE GRADIENT FOR w USING soft_threshold
            # YOUR CODE HERE
            for i in range(d):
                dw[i] = soft_threshold(dw[i],self.alpha)
                
            self.w = self.w - learning_rate * dw
            self.b = self.b - learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        # YOUR CODE HERE
        return X@self.w + self.b

    def loss(self, X_batch, y_batch):
        return mse_loss_vectorized(self.w, self.b, X_batch, y_batch)

In [10]:
model = LassoProximalGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 13896.233224
iteration 10000 / 200000: loss 1324.549732
iteration 20000 / 200000: loss 1280.674386
iteration 30000 / 200000: loss 1395.352993
iteration 40000 / 200000: loss 1427.018531
iteration 50000 / 200000: loss 1635.360648
iteration 60000 / 200000: loss 1334.378618
iteration 70000 / 200000: loss 1385.074723
iteration 80000 / 200000: loss 1368.708933
iteration 90000 / 200000: loss 1336.274407
iteration 100000 / 200000: loss 1379.033838
iteration 110000 / 200000: loss 1385.268406
iteration 120000 / 200000: loss 1435.908773
iteration 130000 / 200000: loss 1292.342501
iteration 140000 / 200000: loss 1330.928126
iteration 150000 / 200000: loss 1328.559512
iteration 160000 / 200000: loss 1407.426431
iteration 170000 / 200000: loss 1468.953572
iteration 180000 / 200000: loss 1545.497176
iteration 190000 / 200000: loss 1299.575273
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2860.189268425761


In [11]:
model = LassoProximalGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 14638.814970
iteration 10000 / 200000: loss 1747.430130
iteration 20000 / 200000: loss 1359.873052
iteration 30000 / 200000: loss 1481.449059
iteration 40000 / 200000: loss 1453.893251
iteration 50000 / 200000: loss 1583.539399
iteration 60000 / 200000: loss 1247.224132
iteration 70000 / 200000: loss 1477.940860
iteration 80000 / 200000: loss 1426.332999
iteration 90000 / 200000: loss 1432.388151
iteration 100000 / 200000: loss 1356.193316
iteration 110000 / 200000: loss 1397.572478
iteration 120000 / 200000: loss 1375.517543
iteration 130000 / 200000: loss 1485.438159
iteration 140000 / 200000: loss 1334.993870
iteration 150000 / 200000: loss 1420.472760
iteration 160000 / 200000: loss 1316.556200
iteration 170000 / 200000: loss 1471.052425
iteration 180000 / 200000: loss 1402.916835
iteration 190000 / 200000: loss 1205.538795
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 2859.8705719977543


# Lasso Projected Gradient Descent

In [12]:
class LassoProjectedGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w_p = None
        self.w_n = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w_p is None: # Initialization
            self.w_p = 0.001 * np.random.randn(d)
            self.w_n = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice,:]
            y_batch = y[choice]
        
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # Project for w_p and w_n
            # YOUR CODE HERE
            """https://github.com/lx10077/lasso/blob/master/project_gradient.py"""
            grad_wp, grad_wn = dw + self.alpha, -dw + self.alpha
            
            self.w_p -= learning_rate * grad_wp
            self.w_n -= learning_rate * grad_wn
            self.w_p[self.w_p <= 0.] = 0.
            self.w_n[self.w_n <= 0.] = 0.
            
            self.b = self.b - learning_rate*db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        return X@(self.w_p - self.w_n) + self.b

    def loss(self, X_batch, y_batch):
        # YOUR CODE HERE
        return mse_loss_vectorized(self.w_p-self.w_n, self.b, X_batch, y_batch)

In [13]:
model = LassoProjectedGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 14197.784293
iteration 10000 / 200000: loss 1597.167252
iteration 20000 / 200000: loss 1487.113621
iteration 30000 / 200000: loss 1440.990709
iteration 40000 / 200000: loss 1417.811269
iteration 50000 / 200000: loss 1391.504646
iteration 60000 / 200000: loss 1325.313098
iteration 70000 / 200000: loss 1491.062523
iteration 80000 / 200000: loss 1488.908711
iteration 90000 / 200000: loss 1473.193459
iteration 100000 / 200000: loss 1520.077644
iteration 110000 / 200000: loss 1525.364734
iteration 120000 / 200000: loss 1444.139781
iteration 130000 / 200000: loss 1258.903449
iteration 140000 / 200000: loss 1329.545745
iteration 150000 / 200000: loss 1447.772623
iteration 160000 / 200000: loss 1525.014201
iteration 170000 / 200000: loss 1524.342257
iteration 180000 / 200000: loss 1461.483184
iteration 190000 / 200000: loss 1302.505875
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2860.299454759736


In [14]:
model = LassoProjectedGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 13378.310039
iteration 10000 / 200000: loss 1295.741053
iteration 20000 / 200000: loss 1561.476330
iteration 30000 / 200000: loss 1330.459293
iteration 40000 / 200000: loss 1330.765235
iteration 50000 / 200000: loss 1499.986134
iteration 60000 / 200000: loss 1481.081293
iteration 70000 / 200000: loss 1395.810184
iteration 80000 / 200000: loss 1283.255696
iteration 90000 / 200000: loss 1348.884619
iteration 100000 / 200000: loss 1519.316045
iteration 110000 / 200000: loss 1438.378284
iteration 120000 / 200000: loss 1360.384265
iteration 130000 / 200000: loss 1465.731064
iteration 140000 / 200000: loss 1562.890857
iteration 150000 / 200000: loss 1352.696134
iteration 160000 / 200000: loss 1503.347190
iteration 170000 / 200000: loss 1524.842182
iteration 180000 / 200000: loss 1435.187928
iteration 190000 / 200000: loss 1374.472346
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 2869.165712153681


# Lasso Coordinate Descent (no intercept)

In [15]:
class LassoCoordinateDescent():
    def __init__(self, alpha=0.1):
        self.w = None
        self.alpha = alpha

    def train(self, X, y, num_iters=1000):
        N, d = X.shape
        
        # YOUR CODE HERE
        """https://github.com/satopirka/Lasso/blob/master/coordinate_descent_lasso.py"""
        beta = np.zeros(d)
        
        for i in range(num_iters):
            for j in range(0,len(beta)):
                tmp_beta = beta.copy()
                tmp_beta[j] = 0.0
               
                arg1 = np.dot(X[:,j], y - np.dot(X,tmp_beta))
                arg2 = self.alpha*N

                beta[j] = soft_threshold(arg1,arg2)/(X[:,j]**2).sum()
            
        self.w = beta
                
    def predict(self, X): 
        # YOUR CODE HERE
        return X.dot(self.w)


In [16]:
model = LassoCoordinateDescent(alpha=0.1)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 26057.118798659812
MSE Coordinate descent model : 26057.116893252627


In [17]:
model = LassoCoordinateDescent(alpha=2)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 28794.88441987604
MSE Coordinate descent model : 28794.885555399043
